In [37]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list((rawArrayDatas[1]))
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list((rawArrayDatas[1][:-forecastDay] ))
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-(mockForcastDay+forecastDay)] & np.log
    ds = rawArrayDatas[0][:-(mockForcastDay+forecastDay)]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-(mockForcastDay+forecastDay)]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of (mockForcastDay+forecastDay)  rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]
    testY= rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출       
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'day')
            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')
    
            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    print('LSTM realforecast :',realForecastDictionary['LSTM'])
    print('Bayseian realforecast :',realForecastDictionary['Bayseian'] ) 
    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
#         listedLogPredict=test_predict[-1].tolist()
#     return [np.exp(y) for y in listedLogPredict]
    return test_predict[-1].tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM testforecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian testforecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm


In [29]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [30]:
columns=['ds','y']
df=pd.read_table('walMonth.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.2)
print(len(df), forecastDay)


33 6


In [31]:
df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

C:\Users\User\Anaconda3\envs\tensorflow35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
rawArrayDatas

[['2010-02-01',
  '2010-03-01',
  '2010-04-01',
  '2010-05-01',
  '2010-06-01',
  '2010-07-01',
  '2010-08-01',
  '2010-09-01',
  '2010-10-01',
  '2010-11-01',
  '2010-12-01',
  '2011-01-01',
  '2011-02-01',
  '2011-03-01',
  '2011-04-01',
  '2011-05-01',
  '2011-06-01',
  '2011-07-01',
  '2011-08-01',
  '2011-09-01',
  '2011-10-01',
  '2011-11-01',
  '2011-12-01',
  '2012-01-01',
  '2012-02-01',
  '2012-03-01',
  '2012-04-01',
  '2012-05-01',
  '2012-06-01',
  '2012-07-01',
  '2012-08-01',
  '2012-09-01',
  '2012-10-01'],
 [32990.769999999997,
  22809.285,
  30103.352000000003,
  16673.537499999999,
  16685.174999999999,
  16383.002,
  16144.702499999999,
  17978.317500000001,
  26928.906000000003,
  23040.349999999999,
  34796.775999999998,
  17286.647499999999,
  31440.657500000001,
  20705.834999999999,
  33011.389999999999,
  17062.93,
  15744.6425,
  15771.246000000001,
  14765.487499999999,
  17551.281999999999,
  24701.7075,
  24634.377499999999,
  34902.413999999997,
  17551.3

In [33]:
answer=LearningModuleRunner(rawArrayDatas,0,forecastDay,'month') #0은 processId

[step: 0] loss: 14.355506896972656
[step: 1] loss: 13.232269287109375
[step: 2] loss: 12.165979385375977
[step: 3] loss: 11.140503883361816
[step: 4] loss: 10.141263961791992
[step: 5] loss: 9.155973434448242
[step: 6] loss: 8.173894882202148
[step: 7] loss: 7.186500549316406
[step: 8] loss: 6.189240455627441
[step: 9] loss: 5.184516429901123
[step: 10] loss: 4.1863813400268555
[step: 11] loss: 3.228079319000244
[step: 12] loss: 2.373772382736206
[step: 13] loss: 1.733634352684021
[step: 14] loss: 1.4662210941314697
[step: 15] loss: 1.6913750171661377
[step: 16] loss: 2.2171549797058105
[step: 17] loss: 2.550845146179199
[step: 18] loss: 2.475527763366699
[step: 19] loss: 2.135228157043457
[step: 20] loss: 1.7567733526229858
[step: 21] loss: 1.4872534275054932
[step: 22] loss: 1.3684765100479126
[step: 23] loss: 1.3732736110687256
[step: 24] loss: 1.4497278928756714
[step: 25] loss: 1.5494227409362793
[step: 26] loss: 1.6382442712783813
[step: 27] loss: 1.6969311237335205
[step: 28] lo

[step: 270] loss: 0.5430952906608582
[step: 271] loss: 0.5346294045448303
[step: 272] loss: 0.5290062427520752
[step: 273] loss: 0.5242760181427002
[step: 274] loss: 0.5176475048065186
[step: 275] loss: 0.5096523761749268
[step: 276] loss: 0.5016870498657227
[step: 277] loss: 0.4942476153373718
[step: 278] loss: 0.48731061816215515
[step: 279] loss: 0.4805811643600464
[step: 280] loss: 0.4746474027633667
[step: 281] loss: 0.47308623790740967
[step: 282] loss: 0.49993348121643066
[step: 283] loss: 0.601479709148407
[step: 284] loss: 0.5748312473297119
[step: 285] loss: 0.44964832067489624
[step: 286] loss: 0.5716143846511841
[step: 287] loss: 0.46086621284484863
[step: 288] loss: 0.5040069818496704
[step: 289] loss: 0.4774826467037201
[step: 290] loss: 0.45776334404945374
[step: 291] loss: 0.48143795132637024
[step: 292] loss: 0.4337202310562134
[step: 293] loss: 0.47327518463134766
[step: 294] loss: 0.4200282394886017
[step: 295] loss: 0.4605700671672821
[step: 296] loss: 0.41309595108

[step: 538] loss: 0.09744267165660858
[step: 539] loss: 0.0972408652305603
[step: 540] loss: 0.09605677425861359
[step: 541] loss: 0.09433252364397049
[step: 542] loss: 0.09342801570892334
[step: 543] loss: 0.09373338520526886
[step: 544] loss: 0.09401610493659973
[step: 545] loss: 0.09316407889127731
[step: 546] loss: 0.09163489192724228
[step: 547] loss: 0.09053961932659149
[step: 548] loss: 0.09014056622982025
[step: 549] loss: 0.08996792882680893
[step: 550] loss: 0.0897412896156311
[step: 551] loss: 0.08949653059244156
[step: 552] loss: 0.08917514979839325
[step: 553] loss: 0.08854030072689056
[step: 554] loss: 0.08768892288208008
[step: 555] loss: 0.08685237169265747
[step: 556] loss: 0.08621063828468323
[step: 557] loss: 0.0856621116399765
[step: 558] loss: 0.08506797254085541
[step: 559] loss: 0.08445703983306885
[step: 560] loss: 0.08393584191799164
[step: 561] loss: 0.08352356404066086
[step: 562] loss: 0.08314673602581024
[step: 563] loss: 0.08277535438537598
[step: 564] los

[step: 793] loss: 0.04221367463469505
[step: 794] loss: 0.041850894689559937
[step: 795] loss: 0.042561694979667664
[step: 796] loss: 0.04212377220392227
[step: 797] loss: 0.04148608446121216
[step: 798] loss: 0.04103603586554527
[step: 799] loss: 0.0411820113658905
[step: 800] loss: 0.04136502742767334
[step: 801] loss: 0.0405580997467041
[step: 802] loss: 0.03996749594807625
[step: 803] loss: 0.04051238298416138
[step: 804] loss: 0.04042293131351471
[step: 805] loss: 0.039589907974004745
[step: 806] loss: 0.03937147185206413
[step: 807] loss: 0.039610493928194046
[step: 808] loss: 0.03941396623849869
[step: 809] loss: 0.03901136666536331
[step: 810] loss: 0.03868287056684494
[step: 811] loss: 0.03859235346317291
[step: 812] loss: 0.03862025961279869
[step: 813] loss: 0.03840318322181702
[step: 814] loss: 0.03795282542705536
[step: 815] loss: 0.03777953237295151
[step: 816] loss: 0.03779929131269455
[step: 817] loss: 0.03763725981116295
[step: 818] loss: 0.03735728934407234
[step: 819

[step: 0] loss: 32.84620666503906
[step: 1] loss: 30.399192810058594
[step: 2] loss: 28.07622528076172
[step: 3] loss: 25.85139274597168
[step: 4] loss: 23.69237518310547
[step: 5] loss: 21.563556671142578
[step: 6] loss: 19.430923461914062
[step: 7] loss: 17.264938354492188
[step: 8] loss: 15.045271873474121
[step: 9] loss: 12.767768859863281
[step: 10] loss: 10.454533576965332
[step: 11] loss: 8.17024040222168
[step: 12] loss: 6.049325942993164
[step: 13] loss: 4.3365654945373535
[step: 14] loss: 3.4211089611053467
[step: 15] loss: 3.7265615463256836
[step: 16] loss: 5.061164855957031
[step: 17] loss: 6.139828681945801
[step: 18] loss: 6.134076118469238
[step: 19] loss: 5.327476501464844
[step: 20] loss: 4.337134838104248
[step: 21] loss: 3.5942859649658203
[step: 22] loss: 3.2418370246887207
[step: 23] loss: 3.223176956176758
[step: 24] loss: 3.4044289588928223
[step: 25] loss: 3.6563451290130615
[step: 26] loss: 3.8869247436523438
[step: 27] loss: 4.043954372406006
[step: 28] loss:

[step: 241] loss: 1.7995173931121826
[step: 242] loss: 1.7891708612442017
[step: 243] loss: 1.8182246685028076
[step: 244] loss: 1.9055407047271729
[step: 245] loss: 1.7912843227386475
[step: 246] loss: 1.8108000755310059
[step: 247] loss: 1.8162797689437866
[step: 248] loss: 1.7536201477050781
[step: 249] loss: 1.8134446144104004
[step: 250] loss: 1.7341547012329102
[step: 251] loss: 1.7936004400253296
[step: 252] loss: 1.7316577434539795
[step: 253] loss: 1.7658039331436157
[step: 254] loss: 1.7334001064300537
[step: 255] loss: 1.7415599822998047
[step: 256] loss: 1.7342950105667114
[step: 257] loss: 1.7237935066223145
[step: 258] loss: 1.7327704429626465
[step: 259] loss: 1.7108652591705322
[step: 260] loss: 1.7285047769546509
[step: 261] loss: 1.7036700248718262
[step: 262] loss: 1.721993327140808
[step: 263] loss: 1.6994993686676025
[step: 264] loss: 1.714137315750122
[step: 265] loss: 1.6972885131835938
[step: 266] loss: 1.7055836915969849
[step: 267] loss: 1.6961867809295654
[st

[step: 481] loss: 0.8446171283721924
[step: 482] loss: 0.8430737257003784
[step: 483] loss: 0.8392200469970703
[step: 484] loss: 0.8318261504173279
[step: 485] loss: 0.8240559697151184
[step: 486] loss: 0.81932532787323
[step: 487] loss: 0.8162580728530884
[step: 488] loss: 0.8142091631889343
[step: 489] loss: 0.8117830753326416
[step: 490] loss: 0.8099356293678284
[step: 491] loss: 0.8095616698265076
[step: 492] loss: 0.8128510117530823
[step: 493] loss: 0.8274366855621338
[step: 494] loss: 0.8579116463661194
[step: 495] loss: 0.9308547973632812
[step: 496] loss: 0.9870738387107849
[step: 497] loss: 1.0199291706085205
[step: 498] loss: 0.8489252328872681
[step: 499] loss: 0.7681407928466797
[step: 500] loss: 0.8473777174949646
[step: 501] loss: 0.8903800845146179
[step: 502] loss: 0.827391505241394
[step: 503] loss: 0.756142258644104
[step: 504] loss: 0.8108535408973694
[step: 505] loss: 0.8601551055908203
[step: 506] loss: 0.780002236366272
[step: 507] loss: 0.7503805160522461
[step:

[step: 713] loss: 0.5643657445907593
[step: 714] loss: 0.5270862579345703
[step: 715] loss: 0.5503684282302856
[step: 716] loss: 0.5417195558547974
[step: 717] loss: 0.5267772078514099
[step: 718] loss: 0.5470768213272095
[step: 719] loss: 0.5257828235626221
[step: 720] loss: 0.5324280858039856
[step: 721] loss: 0.5355688333511353
[step: 722] loss: 0.521119236946106
[step: 723] loss: 0.5329208970069885
[step: 724] loss: 0.5252481698989868
[step: 725] loss: 0.5209692716598511
[step: 726] loss: 0.5291329622268677
[step: 727] loss: 0.5186219811439514
[step: 728] loss: 0.5210577249526978
[step: 729] loss: 0.5227869749069214
[step: 730] loss: 0.5152958631515503
[step: 731] loss: 0.5192562341690063
[step: 732] loss: 0.5176116824150085
[step: 733] loss: 0.5131399631500244
[step: 734] loss: 0.5167831182479858
[step: 735] loss: 0.5134488940238953
[step: 736] loss: 0.5114387273788452
[step: 737] loss: 0.5136173963546753
[step: 738] loss: 0.5104324817657471
[step: 739] loss: 0.5093684196472168
[s

[step: 936] loss: 0.3929312825202942
[step: 937] loss: 0.3922739624977112
[step: 938] loss: 0.39163100719451904
[step: 939] loss: 0.39098480343818665
[step: 940] loss: 0.3903258442878723
[step: 941] loss: 0.389664888381958
[step: 942] loss: 0.3890111446380615
[step: 943] loss: 0.3883611857891083
[step: 944] loss: 0.3877057433128357
[step: 945] loss: 0.387043297290802
[step: 946] loss: 0.3863816261291504
[step: 947] loss: 0.3857230544090271
[step: 948] loss: 0.3850656747817993
[step: 949] loss: 0.38440483808517456
[step: 950] loss: 0.3837393820285797
[step: 951] loss: 0.38307294249534607
[step: 952] loss: 0.3824089765548706
[step: 953] loss: 0.38174498081207275
[step: 954] loss: 0.38107919692993164
[step: 955] loss: 0.38041040301322937
[step: 956] loss: 0.37973886728286743
[step: 957] loss: 0.3790682256221771
[step: 958] loss: 0.3783974349498749
[step: 959] loss: 0.3777257204055786
[step: 960] loss: 0.3770526051521301
[step: 961] loss: 0.37637731432914734
[step: 962] loss: 0.37570095062

In [34]:
len(answer)

6

In [35]:
answer

[18505.381228343118,
 17436.523430305366,
 19748.680869264314,
 18776.571383034858,
 23150.508624040402,
 29887.249859914671]

In [36]:
len(rawArrayDatas[1])

33